In [ ]:
# Execute if necessary
# %%capture
# !pip install pandas

In [ ]:
import pandas as pd

# Práctica 7: Modelos Integrados (ARIMA y SARIMA)

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones.

La entrega de la práctica será en la siguiente sesión a menos que la indicación sea otra. La revisión iniciará al iniciar la sesión y únicamente podrá ser evaluada durante la duración de la sesión.

__Para esta práctica se deben usar bibliotecas__. Se recomienda el uso de:

- scikit-learn (https://scikit-learn.org/stable/)
- plotly express (https://plotly.com/python/plotly-express/)
- statsmodels (https://www.statsmodels.org/stable/index.html).

## Asignación 1

Seleccione algun dataset de su preferencia que sea relacionado a series de tiempo. Particione sus datos para poder evaluar el desempeño de su práctica (__conjunto de test__).

In [ ]:
!pip install nbformat>=4.2.0


In [ ]:
!pip install plotly

In [ ]:
#importar csv kalimati_tarkari_dataset.csv y visualizarlo
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

df = pd.read_csv('kalimati_tarkari_dataset.csv')
df.head()

In [ ]:
#Seleccionar en la columna Commodity las entradas con el valor 'Potato White' y visualizarlo
df = df[df['Commodity'] == 'Onion Dry (Indian)']
df.head(20)

In [ ]:
df_med = df[['Date', 'Average']]
train = df_med.loc['2013-06-16':'2021-03-01']
test = df_med.loc['2021-03-01':'2021-05-13']


## Asignación 2

Realice un análisis exploratorio de datos y el preprocesamiento necesario para el dataset seleccionado. El análisis realizado debe respaldar (preferentemente mediante el uso de gráficas) las decisiones que tome para la siguiente asignación.

In [ ]:
#Seleccionar las columnas Date, Average y visualizarlo
df_med = df[['Date', 'Average']]
df_low = df[['Date', 'Minimum']]
df_high = df[['Date', 'Maximum']]

df_pred = df_med.copy()

df_med.head()

#graficar con plotly-express los 3 dataframes df_med, df_low y df_high 
fig = px.line(df_med, x='Date', y='Average', title='Average')
fig.show()

In [ ]:
df_med.head()

In [ ]:
#dividir el dataframe df_med en 2 dataframes: df_train y df_test en 80% y 20% respectivamente

X = df_med.iloc[:, :-1].values
y = df_med.iloc[:, 1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Asignación 3

Ajuste un modelo ARMA, ARIMA o SARIMA segun sea conveniente para sus datos.

Para poder evaluar que modelo nos conviene más optamos por seguir los siguientes pasos: 

Aquí hay algunos pasos que puedes seguir para evaluar qué modelo es más apropiado:

- __Visualización de datos:__ Comienza por trazar la serie temporal de los datos que tienes, es decir, los valores promedio en función de la fecha. Esto te permitirá observar cualquier patrón evidente, tendencia o estacionalidad.

- __Estacionariedad:__ Evalúa si la serie temporal es estacionaria. Una serie estacionaria es aquella cuyas propiedades estadísticas (como media y varianza) son constantes en el tiempo. Puedes utilizar métodos como la prueba de Dickey-Fuller aumentada (ADF) para verificar la estacionariedad.


- __Autocorrelación:__ Examina la autocorrelación de la serie temporal. La autocorrelación es la relación entre los valores pasados y futuros en una serie temporal. Puedes utilizar la función de autocorrelación (ACF) y la función de autocorrelación parcial (PACF) para evaluar la correlación en diferentes rezagos.
Componentes estacionales: Si observas patrones estacionales en los datos, como fluctuaciones regulares que se repiten, puedes considerar modelos como SARIMA que incorporan componentes estacionales.

### 1.- Visualizacion de datos

Visualizando la anterior grafica podemos observar que existe un patrón de fechas en finales de año

### 2.- Estacionariedad

Cuando se realiza la prueba de Dickey-Fuller Aumentada (ADF), el objetivo es determinar si una serie temporal tiene una raíz unitaria, lo que indica la presencia de tendencia o no estacionariedad en los datos. La hipótesis nula en esta prueba es que la serie temporal tiene una raíz unitaria, es decir, no es estacionaria.

Al realizar la prueba ADF, se obtiene un estadístico ADF específico que se compara con los valores críticos correspondientes a diferentes niveles de significancia, como el 1%, 5% y 10%. Estos valores críticos representan los umbrales a los que se compara el estadístico ADF para determinar si se rechaza o no la hipótesis nula.

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Extrae la columna de interés (Average en este caso)
average_data = df_med['Average']

# Aplica la prueba Dickey-Fuller aumentada
result = adfuller(average_data)

# Extrae los valores de la prueba y los p-valores
adf_statistic = result[0]
p_value = result[1]
critical_values = result[4]

# Imprime los resultados
print(f'Estadístico ADF: {adf_statistic}')
print(f'p-valor: {p_value}')
print('Valores críticos:')
for key, value in critical_values.items():
    print(f'   {key}: {value}')

El valor del estadístico ADF (-3.430956211048367) es menor que el valor crítico del 1% (-3.432750244284443). Esto significa que se rechaza la hipótesis nula al 1% de nivel de significancia. En otras palabras, hay evidencia suficiente para concluir que los datos no tienen una raíz unitaria y son estacionarios al 1% de nivel de significancia. Esta es una conclusión bastante sólida.

Sin embargo, el valor del estadístico ADF es mayor que los valores críticos del 5% (-2.8626004539809906) y del 10% (-2.567334514143628). Esto implica que no se puede rechazar la hipótesis nula al 5% y 10% de nivel de significancia. En otras palabras, no hay suficiente evidencia para afirmar que los datos son estacionarios al 5% y 10% de nivel de significancia.

In [ ]:
df_med['Diferencia entre precios'] = df_med['Average'] - df_med['Average'].shift(1)
df_med.dropna(subset = ["Diferencia entre precios"], inplace=True)
df_med.head()

In [ ]:
ad_fuller_result = adfuller(df_med['Diferencia entre precios'])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
plt.figure(figsize=[12, 5]); # Set dimensions for figure
df_med['Diferencia entre precios'].plot()
plt.title('Precio promedio de la cebolla en el mercado de Kalimati - Diferencia entre precios')
plt.ylabel('Precio')
plt.grid(True)

### 3.-Autocorrelacion



La función tsaplots en statsmodels.graphics de Python proporciona varias tramas (plots) para el análisis y visualización de series de tiempo. Esta función se utiliza en combinación con otros módulos y funciones de statsmodels para crear gráficos relacionados con el análisis de series temporales.

In [ ]:
!pip install --upgrade matplotlib


In [ ]:
#realiza una grafica de autocorrelacion con plotly-express de la columna Average del dataframe df_med y la de autocorrelacion parcial
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(df_med['Average'], lags=1000)
plot_pacf(df_med['Average'], lags=1000)

- Modelos ARIMA y SARIMA: Si la serie es estacionaria y no muestra patrones estacionales evidentes, puedes considerar modelos ARIMA. Un modelo ARIMA es adecuado cuando hay una dependencia de los valores pasados y diferencias en los datos para hacerlos estacionarios. Si hay componentes estacionales, puedes considerar modelos SARIMA que extienden ARIMA para manejar la estacionalidad.

- Modelo ARMA: Si no hay necesidad de diferenciar los datos para hacerlos estacionarios y no hay patrones estacionales, un modelo ARMA puede ser apropiado. El modelo ARMA combina los componentes autorregresivos (AR) y de media móvil (MA) para modelar la dependencia y la estructura de ruido en la serie temporal.

Por lo que determinamos que utilizaremos un modelo __SARIMA__ para poder realizar nuestro forecasting.

Nuestro modelo entrenado luce de esta manera.

añadir aqui

In [ ]:
df = pd.read_csv('kalimati_tarkari_dataset.csv')
df.head()
#Seleccionar en la columna Commodity las entradas con el valor 'Potato White' y visualizarlo
df = df[df['Commodity'] == 'Onion Dry (Indian)']
df_med = df[['Date', 'Average']]

#hacer index la columna Date y convertirlo en formato fecha
df_med['Date'] = pd.to_datetime(df_med['Date'])
df_med.set_index('Date', inplace=True)
df_med.head()
df_med2 = df_med.copy()

In [ ]:
!pip install cufflinks

In [ ]:
# Import Plotly & Cufflinks libraries and run it in Offline mode
import plotly.offline as py
import cufflinks as cf
py.init_notebook_mode(connected=True)
py.enable_mpl_offline()
cf.go_offline()

In [ ]:
df_med.iplot(title="Valor de la cebolla en el mercado de Kalimati")

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

#dejar que Date sea DateTimeIndex

df_med.index = pd.to_datetime(df_med.index)
df_med = df_med.asfreq('d')

In [ ]:
print(df_med.index.freq)

In [ ]:
df_med = df_med.fillna(df_med.mean())

In [ ]:
result = seasonal_decompose(df_med, model='multiplicative')
fig = result.plot()

Esta función se utiliza para encontrar automáticamente los parámetros óptimos de un modelo ARIMA (Autoregressive Integrated Moving Average) para una serie de tiempo.

El modelo ARIMA es un enfoque estadístico utilizado para modelar y predecir datos de series de tiempo. Consiste en tres componentes principales: el componente autoregresivo (AR), el componente de media móvil (MA) y el componente de integración (I).

En el código que has proporcionado, se crea un modelo ARIMA utilizando la función auto_arima(). A continuación se explican los parámetros utilizados en la función:

- df_med: Es la serie de tiempo a la que se le desea ajustar el modelo ARIMA.
- start_p y start_q: Son los valores iniciales para los parámetros autoregresivos (p) y de media móvil (q), respectivamente.
- max_p y max_q: Son los valores máximos permitidos para los parámetros p y q. Estos parámetros limitan la búsqueda de los mejores valores en un rango específico.
- m: Es la frecuencia de la serie de tiempo, que en este caso se establece en 7 para indicar una serie semanal.
- start_P: Es el valor inicial para el parámetro de autoregresión estacional (P).
- seasonal: Indica si se debe utilizar un modelo estacional ARIMA.
- d y D: Son los órdenes de diferenciación para los componentes no estacionales y estacionales, respectivamente.
- trace: Indica si se debe imprimir información de depuración durante la búsqueda de los mejores parámetros.
- error_action y suppress_warnings: Son configuraciones para manejar errores y advertencias.
- stepwise: Indica si se debe utilizar un enfoque paso a paso para buscar los mejores parámetros.

Al ejecutar este código, se obtendrá un modelo ARIMA ajustado a la serie de tiempo df_med con los mejores parámetros encontrados automáticamente según los criterios especificados.

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima
stepwise_model = auto_arima(df_med, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

In [ ]:
print(stepwise_model.aic())

Seleccionamos nuestro valores de prueba

In [ ]:
train = df_med2.loc['2013-06-16':'2021-03-01']
test = df_med2.loc['2021-03-01':'2021-05-13']

Ajustamos el modelo

In [ ]:
stepwise_model.fit(train)

## Asignación 4

Realice un diagnostico de su modelo para verificar que es correcto.

In [ ]:
future_forecast = stepwise_model.predict(n_periods=73)
print(future_forecast)

In [ ]:
future_forecast = pd.DataFrame(future_forecast, index=test.index, columns=['Prediction'])

In [ ]:
pd.concat([test, future_forecast], axis=1).iplot()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Obtener las predicciones del modelo
predictions = stepwise_model.predict(n_periods=len(test))

# Calcular métricas de evaluación
mse = mean_squared_error(test, predictions)
mae = mean_absolute_error(test, predictions)

In [ ]:
#imprimir las metricas de evaluacion 
print(f'MSE: {mse}')
print(f'MAE: {mae}')

## Asignación 5

Obtenga el forecast para los datos de test, grafique los valores reales y las predicciones en conjunto. Obtenga el error de regresión para los datos de test.

In [ ]:
stepwise_model.fit(df_med)

In [ ]:
future_forecast_1week = stepwise_model.predict(n_periods=30)

In [ ]:
print(future_forecast_1week)

In [ ]:
!pip install -U kaleido


In [ ]:
import matplotlib.pyplot as plt

# Crear figura y ejes
fig, ax = plt.subplots(figsize=(12, 6))

# Graficar la serie de tiempo original
ax.plot(df_med.index, df_med['Average'], label='Serie de tiempo original')

# Graficar las predicciones
ax.plot( future_forecast_1week, label='Predicciones')

# Configurar límites del eje x
start_date = pd.to_datetime('2021-02-01')
end_date = pd.to_datetime('2021-07-01')
ax.set_xlim(start_date, end_date)

# Configurar etiquetas y título
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.title('Predicciones para la próxima semana')

# Rotar las etiquetas del eje x para mejorar la legibilidad
plt.xticks(rotation=45)

# Mostrar la leyenda
plt.legend()

# Mostrar la gráfica
plt.show()


In [ ]:
from datetime import datetime, timedelta
from datetime import date
fecha_inicio = datetime.strptime("2021-05-13", "%Y-%m-%d")
diferencia = timedelta(days=182)
fecha_resultante = fecha_inicio - diferencia

print(fecha_resultante.strftime("%Y-%m-%d"))

fecha_inicio = date(2021, 3, 1)
fecha_fin = date(2021, 5, 13)
diferencia = fecha_fin - fecha_inicio

print(diferencia.days)
